Use a pretrained network to extract features
    + Load the pretrained model.
    + Create numpy arrays and fill them by running data through  pretrained model.
    + 

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

### load pretrained model

In [2]:
from keras.applications import VGG16

c:\users\a00439512\appdata\local\continuum\anaconda3\envs\demo\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
conv_base_model = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))
conv_base_model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 150, 150, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0         
__________

### SetUp Data sources

In [4]:
import os
import numpy as np

from keras.preprocessing.image import ImageDataGenerator

In [5]:
pwd_ = os.getcwd()
pwd_

'D:\\Github\\DeepNets'

In [6]:
path = os.path.dirname(os.path.dirname(pwd_))
print(path)

D:\


In [7]:
dest_data_folder = os.path.join(os.path.join(path, "Warehouse"), "cats_and_dogs")

train_dir = os.path.join(dest_data_folder, "train")
cat_train_dir = os.path.join(train_dir, "cat")
dog_train_dir = os.path.join(train_dir, "dog")


test_dir = os.path.join(dest_data_folder, "test")
cat_test_dir = os.path.join(test_dir, "cat")
dog_test_dir = os.path.join(test_dir, "dog")


validation_dir = os.path.join(dest_data_folder, "validation")
cat_validation_dir = os.path.join(validation_dir, "cat")
dog_validation_dir = os.path.join(validation_dir, "dog")


### Extracting features using the pretrained convolutional base

In [8]:
train_DataGen = ImageDataGenerator(rescale=1.0/255)
train_DG = train_DataGen.flow_from_directory(
            train_dir,
            target_size=(150, 150),
            batch_size=32,
            class_mode='binary')


Found 4000 images belonging to 2 classes.


In [9]:
# for (batch_data,batch_label) in train_DG:
#        print(batch_label.shape)

In [10]:
batch_size = 2000

def extract_features(directory, num_of_samples):
    
    features = np.zeros(shape=(num_of_samples, 4, 4, 512))
    labels = np.zeros(shape=(num_of_samples,))
    
    train_DG = train_DataGen.flow_from_directory(
            train_dir,
            target_size=(150, 150),
            batch_size=num_of_samples,
            class_mode='binary')
    
    i = 0
    for (batch_data,batch_label) in train_DG:
        base_features = conv_base_model.predict(batch_data)
        
        features[i*(batch_size): (i+1)*(batch_size)] = base_features
        labels[i*(batch_size): (i+1)*(batch_size)] = labels
        
        i = i+1
        
        if (i*(batch_size) > num_of_samples):
            break;
            
    return features, labels


In [11]:
train_features, train_labels = extract_features(train_dir, 2000)
validation_features, validation_labels = extract_features(validation_dir, 1000)
test_features, test_labels = extract_features(test_dir, 1000)

Found 4000 images belonging to 2 classes.


KeyboardInterrupt: 

In [ ]:
train_features.shape
train_labels.shape

In [ ]:
# flatten the feature sets

train_features = np.reshape(train_features, (2000, 4 * 4 * 512))
validation_features = np.reshape(validation_features, (1000, 4 * 4 * 512))
test_features = np.reshape(test_features, (1000, 4 * 4 * 512))

### Simple feed forward

In [ ]:
from keras import models
from keras import layers
from keras import optimizers

model = models.Sequential()

model.add(layers.Dense(256, activation='relu', input_dim=4 * 4 * 512))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer=optimizers.RMSprop(lr=2e-5),
loss='binary_crossentropy',
metrics=['acc'])


history = model.fit(train_features, train_labels,
epochs=30,
batch_size=20,
validation_data=(validation_features, validation_labels))

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
    
plt.show()